# Import Dependencies

In [186]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize

from matplotlib import pyplot as plt

from pathlib import Path
from tqdm import tqdm

import warnings

warnings.filterwarnings('ignore')

# Load Data

In [ ]:
# industry returns
df = pd.read_csv(str(Path().absolute()) + "/data/48_Industry_Portfolios.CSV", index_col = 0, skiprows = 11, nrows = 1182, header=0)
df.index = pd.to_datetime(df.index, format = "%Y%m")
df = df / 100

# remove NAs
mask = (df <= -0.99)
df[mask] = np.nan

# nb of industries dataframe
nb_industries = pd.read_csv(str(Path().absolute()) + "/data/48_Industry_Portfolios.CSV", index_col = 0, skiprows = 2587, nrows = 1182, header=0)
nb_industries.index = pd.to_datetime(nb_industries.index, format = "%Y%m")
mask = (nb_industries <= -0.99)
nb_industries[mask] = np.nan

# average sizes dataframe
avg_size = pd.read_csv(str(Path().absolute()) + "/data/48_Industry_Portfolios.CSV", index_col = 0, skiprows = 3773, nrows = 1182, header=0)
avg_size.index = pd.to_datetime(avg_size.index, format = "%Y%m")
mask = (avg_size <= -0.99)
avg_size[mask] = np.nan

# sum of BE / sum of ME dataframe
be_over_me = pd.read_csv(str(Path().absolute()) + "/data/48_Industry_Portfolios.CSV", index_col = 0, skiprows = 4959, nrows = 99, header=0)
be_over_me.index = pd.to_datetime(be_over_me.index, format = "%Y")

In [ ]:
# market cap of each industry over time
mkt_cap = nb_industries * avg_size

# momentum with monthly data
momentum = df.rolling(12).mean()

# book value to market value
# resample be_over_me to monthly data
# we must first shift years since our "factor year" begins in July preventing us from grouping by years
be_over_me.index = be_over_me.index + pd.DateOffset(months = 6)
be_over_me = be_over_me.resample("MS").ffill()

# need to add missing portion of 2024 since data with shifted index is missing it
extra_be_over_me = pd.DataFrame(np.repeat([be_over_me.iloc[-1].values], repeats = 5, axis = 0), 
                                index = pd.date_range(start=be_over_me.index[-1] + pd.DateOffset(months = 1), end='2024-12-01', freq='MS'), 
                                columns = be_over_me.columns)

be_over_me = pd.concat([be_over_me, extra_be_over_me], axis = 0)
mask = (be_over_me <= -99.99)
be_over_me[mask] = np.nan

(1182, 48)
(1177, 48)
(1182, 48)


# Normalize Data

In [189]:
mkt_cap_ = mkt_cap.loc['1927-06-01':'1973-12-01']
mkt_cap_norm = (mkt_cap_ - mkt_cap_.mean()) / mkt_cap_.std()
print(mkt_cap_norm.shape)

be_over_me_ = be_over_me.loc['1927-06-01':'1973-12-01']
be_over_me_norm = (be_over_me_ - be_over_me_.mean()) / be_over_me_.std()
print(be_over_me_norm.shape)

momentum_ = momentum.loc['1927-06-01':'1973-12-01']
momentum_norm = (momentum_ - momentum_.mean()) / momentum_.std()
print(momentum_norm.shape)

df_in = df.loc['1927-06-01':'1973-12-01']
print(df_in.shape)

(559, 48)
(559, 48)
(559, 48)
(559, 48)


In [ ]:
def CRRA(wealth: float, gamma = 5):
    """"
    Constant Relative Risk Aversion Utility Function
    ---
    :param wealth: current wealth level of investor
    :param gamma: risk aversion parameter
    :return: CRRA utility level as given by functional form in Brandt et al. (2009), equation 15
    """

    if gamma == 1:
        return np.log(wealth)
    else:
        return ((1 + wealth) ** (1 - gamma)) / (1 - gamma)


In [ ]:
characteristics = np.stack([mkt_cap_norm, be_over_me_norm, momentum_norm], axis= -1)  # 3 characteristics we're interested in
weights = mkt_cap.iloc[-1]/ mkt_cap.iloc[-1].sum()  # weights of market portfolio (our benchmark)
theta = np.array([-1.451, 3.606, 1.772])  # initial guess for theta

# Optimization

## Objective

In [ ]:
def objective(theta:np.ndarray, x:np.ndarray, rets:pd.DataFrame, weights:np.ndarray):
    """
    
    """
    
    accrued_wealth = 0.0
    wealth = 0.0
    for t in range(x.shape[0]):
        rets_t1 = rets.iloc[t+1, :].values 
        x_t = x[t, :, :]
        
        valid_mask = ~np.isnan(rets_t1) & ~np.isnan(x_t).any(axis=1)
        Nt = valid_mask.sum()
        
        if Nt > 0:
            wealth += np.sum((weights[valid_mask] + (x_t[valid_mask] @ theta) / Nt) * rets_t1[valid_mask])
        accrued_wealth += CRRA(wealth)
    
    return - accrued_wealth / x.shape[0]

In [193]:
init = np.array([-1.398e+01,  4.366e+01, 2.770e+01]) # local solution found
# init = theta
response = minimize(objective, x0= init, args= (characteristics, df, weights), method= 'SLSQP')
response

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 6.494698443758094e-06
       x: [-1.398e+01  4.366e+01  2.770e+01]
     nit: 1
     jac: [ 5.138e-07 -4.033e-06  5.735e-06]
    nfev: 4
    njev: 1

# Find New Weights

In [194]:
next_w = np.zeros(weights.shape)
for i in range(len(weights)):
    next_w[i] = weights[i] + (response.x @ characteristics[-1,i,:]) / (np.count_nonzero(~np.isnan(characteristics[-1,:,:]))/3)

next_w

array([-0.15392515, -1.63226366, -2.58114127, -2.3781702 , -2.01743452,
       -1.73431925, -2.48397976, -2.06254471, -2.80009338, -1.48297405,
        1.91309059, -2.8094114 , -2.42926598, -0.22485848, -1.61746641,
       -1.46347063, -0.9922424 , -1.85080179,  0.1245534 , -1.54020126,
       -1.33436336, -1.30295921, -0.76151226, -0.1109189 , -0.20995246,
        1.19035235,  0.43982617, -0.34550187, -0.7077866 , -1.19528758,
       -0.97132281, -0.50712861, -1.87492782, -2.09387426, -2.02236567,
       -1.11023043, -1.26536404, -0.7541866 , -1.89966462, -1.4040221 ,
       -2.04357981, -2.08440924, -2.84235124, -2.03303118, -1.6115431 ,
       -2.03087226, -2.68606177, -4.46803079])

In [195]:
# apply equation 16
def long_only_constraint(weights):
    w_pos = np.clip(weights, 0, None) # set all negative to 0
    return w_pos / np.sum(w_pos)

l_w = long_only_constraint(next_w)
l_w

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.52158756, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.0339584 , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.32453925, 0.11991479, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        ])

# Out-of-Sample Testing

### 0. Load Out-of-Sample Data

In [196]:
mkt_cap_ = mkt_cap.loc['1963-12-01':'2024-01-01']
mkt_cap_norm = (mkt_cap_ - mkt_cap_.mean()) / mkt_cap_.std()
print(mkt_cap_norm.shape)

be_over_me_ = be_over_me.loc['1963-12-01':'2024-01-01']
be_over_me_norm = (be_over_me_ - be_over_me_.mean()) / be_over_me_.std()
print(be_over_me_norm.shape)

momentum_ = momentum.loc['1963-12-01':'2024-01-01']
momentum_norm = (momentum_ - momentum_.mean()) / momentum_.std()
print(momentum_norm.shape)

df_out = df.loc['1963-12-01':'2024-03-01']
print(df_out.shape)


(722, 48)
(722, 48)
(722, 48)
(724, 48)


In [197]:
x_hat = np.stack([mkt_cap_norm, be_over_me_norm, momentum_norm], axis= -1)
w_bar = mkt_cap.loc['1973-12-01']/ mkt_cap.loc['1973-12-01'].sum()
theta = response.x
theta

array([-13.98,  43.66,  27.7 ])

### Rolling Window

In [198]:
window_size = 120 # 10 years x 12 months = 120 timesteps
next_w = np.zeros(w_bar.shape[0])
monthly_rets = []

for t in tqdm(range(window_size, x_hat.shape[0])): 
    x_hat_subset = x_hat[t-window_size:t, :, :]
    df_out_subset = df_out.iloc[t-window_size:t+1]
    w_bar = mkt_cap_.iloc[t]/ mkt_cap_.iloc[t].sum()

    
    # 1. Estimate Theta
    res = minimize(objective, x0= theta, args= (x_hat_subset, df_out_subset, w_bar), method= 'SLSQP')
    theta = res.x
    # 2. Estimate Weights
    denom = np.count_nonzero(~np.isnan(x_hat_subset[-1, :, :])) / 3
    valid = ~np.isnan(x_hat_subset[-1, :, :]).any(axis=1)
    next_w[valid] = weights[valid] + (x_hat_subset[-1, valid, :] @ theta) / denom

    long_weights = long_only_constraint(next_w)
    # 3. Estimate Next Month Returns
    rets_clean = np.nan_to_num(df_out.iloc[t+1], nan=0)
    # 4. Record Return
    monthly_rets.append(long_weights @ rets_clean)

100%|██████████| 602/602 [06:16<00:00,  1.60it/s]


In [206]:
clean_rets = [r for r in monthly_rets if not np.isnan(r)]
print(f'mean: {np.mean(clean_rets)}')
print(f'std: {np.std(clean_rets)}')
print(f'sharpe: {np.mean(clean_rets)/np.std(clean_rets)}')

mean: 0.008217644528510122
std: 0.05762429226886323
sharpe: 0.1426072964188135
